In [1]:
import math
import re
import sys
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve,roc_auc_score

# device =torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:0')
device = torch.device('cpu')

In [2]:
ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-10.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-45.csv')
ori_fea = ori_fea.drop(ori_fea.columns[0], axis=1)

label = [1] * 8000 + [0] * 8000

oy = pd.DataFrame(label, columns=['label'])

ori_fea

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.380952,315,0.346032,0.155556,0.225397,0.273016,0.169329,0.015974,0.089457,0.070288,...,1.739852,1.488545,2.126691,1.638874,1.959795,1.824955,1.203994,2.112745,1.521891,1.287371
1,0.409396,596,0.261745,0.177852,0.231544,0.328859,0.104377,0.042088,0.055556,0.060606,...,1.025578,1.778595,1.755717,1.525106,1.711237,1.714325,1.498462,1.806120,1.505074,1.592989
2,0.343101,819,0.312576,0.191697,0.151404,0.344322,0.105263,0.059976,0.048960,0.097919,...,1.762870,1.456490,1.617004,1.670802,1.524033,1.769670,0.861199,1.557655,1.637078,1.445257
3,0.461774,327,0.244648,0.299694,0.162080,0.293578,0.080000,0.058462,0.036923,0.067692,...,1.109550,2.209669,2.012371,1.320911,1.674553,1.648623,1.475377,1.641878,1.688234,1.039242
4,0.565217,23,0.130435,0.347826,0.217391,0.304348,0.047619,0.000000,0.000000,0.095238,...,1.092264,1.672603,1.600446,1.095031,1.490806,1.444283,1.125027,1.053864,1.945725,1.240614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.450256,975,0.274872,0.217436,0.232821,0.274872,0.069887,0.045221,0.073998,0.085303,...,1.644713,1.535963,1.624135,1.270853,1.732259,2.396312,1.253346,1.316512,2.523425,0.971581
15996,0.430957,1557,0.289017,0.185613,0.245344,0.280026,0.093248,0.041801,0.079100,0.074598,...,1.933401,1.304199,1.850107,1.453508,1.252683,2.442782,1.229007,1.140442,2.146400,1.446444
15997,0.455010,2934,0.311179,0.190184,0.264826,0.233810,0.106753,0.042633,0.103342,0.058322,...,1.464233,1.847922,1.433326,1.518303,2.239937,2.438138,1.361260,2.233932,2.111112,1.483973
15998,0.448404,2694,0.293615,0.213066,0.235338,0.257981,0.089525,0.056092,0.080609,0.067608,...,2.016628,1.108685,1.966902,1.689092,1.072085,1.463851,1.365601,1.386257,1.749610,1.083975


In [45]:
def extract_sequences_from_fasta(file_path):
    headers = []
    sequences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_header = ''
        current_sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_header and current_sequence:
                    headers.append(current_header)
                    sequences.append(current_sequence)

                current_header = line[1:]
                current_sequence = ''
            else:
                current_sequence += line

        # 处理最后一个序列
        if current_header and current_sequence:
            headers.append(current_header)
            sequences.append(current_sequence)

    data = {'seqname': headers, 'seq': sequences}
    df = pd.DataFrame(data)
    return df

# 读取FASTA文件并提取序列
file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\cRNA\Zea mays.fasta'
datah = extract_sequences_from_fasta(file_path)
print(datah.shape[0])

file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\ncRNA\Zea mays.fasta'
datah_nc = extract_sequences_from_fasta(file_path)
print(datah_nc.shape[0])
datah = pd.concat([datah,datah_nc]).reset_index(drop=True)

pattern = r'(?:^|\s)([^ ]+)'  # 匹配第一个空格前的非空格字符序列作为标识部分
datah['seqname'] = [re.search(pattern, identifier).group(1) for identifier in datah['seqname']]

data = datah
datah

7406
7406


,seqname,seq
0,Zm00001d033646_T002,ATGGCCGCCGCGACAGAGAAGACGGCTGAGGACATCCGCCGCGAGC...
1,Zm00001d002782_T001,ATGGCCTCTCCTTCCCCTTCTTCCCCCGCCCGCGCCTCCGGCCGCC...
2,Zm00001d007357_T004,ATGTTTCTTAAGGAACTAGACTTACAAAAGAGTTGTGTGAAACATC...
3,Zm00001d008727_T002,ATGGCTCAGATCTTGCTCCACGGCACGCTCCACGCCACCATCTTCG...
4,Zm00001d003476_T001,ATGCACCAGGATGCACACGAGTTCTTAAATTTTCTTTTGAATGAAC...
...,...,...
14807,URS0001BC9443_4577,CUCCAAAGGGAUCGCAUUGAUCUAACCUGCCGAUCGACGCCGACGU...
14808,URS00021F59AE_4577,GGGGAAAGTCTTTGACTTGCTAGCCAGCCGTAGTAGTAATAATACG...
14809,URS00021F600D_4577,ACTTACCTGGACGGGGTCGACGGGCTATCAAGAAGGCCCGTGGCCT...
14810,URS00021F6599_4577,TCTTGCCTTGCGCGATCTAAGTCTAAGGAAATACAAAAATGCGCTT...


In [46]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Zea_mays',names=names,sep='\t')
print(rna_cds.shape[0])
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Zea_mays',names=names,sep='\t')
print(rna_cds_nc.shape[0])

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)

datah_seqname = datah['seqname']
rna_cds_seqname = rna_cds['seqname']

# # extra_rows_index = datah_seqname.difference(rna_cds_seqname)
indexes_to_drop = datah_seqname[~datah_seqname.isin(rna_cds_seqname)].index

# 删除行
datah.drop(indexes_to_drop, inplace=True)

# 创建一个包含每个打印的元素的列表
label_data = [0] * 7398 + [1] * 6377

# 使用pandas库将列表转换为DataFrame
y = pd.DataFrame(label_data, columns=['label'])
datah.reset_index(drop=True, inplace=True)

datah.insert(0,'label',y)
print(datah.index)
print(y.index)

data3 = datah.copy()
datah

7398
6377
RangeIndex(start=0, stop=13775, step=1)
RangeIndex(start=0, stop=13775, step=1)


,label,seqname,seq
0,0,Zm00001d033646_T002,ATGGCCGCCGCGACAGAGAAGACGGCTGAGGACATCCGCCGCGAGC...
1,0,Zm00001d002782_T001,ATGGCCTCTCCTTCCCCTTCTTCCCCCGCCCGCGCCTCCGGCCGCC...
2,0,Zm00001d007357_T004,ATGTTTCTTAAGGAACTAGACTTACAAAAGAGTTGTGTGAAACATC...
3,0,Zm00001d008727_T002,ATGGCTCAGATCTTGCTCCACGGCACGCTCCACGCCACCATCTTCG...
4,0,Zm00001d003476_T001,ATGCACCAGGATGCACACGAGTTCTTAAATTTTCTTTTGAATGAAC...
...,...,...,...
13770,1,URS0000DA242E_4577,ATACATTTCTTGGCCTTTGGCTAAGATAAAATCTAATAGTTGTTTT...
13771,1,URS00021F59AE_4577,GGGGAAAGTCTTTGACTTGCTAGCCAGCCGTAGTAGTAATAATACG...
13772,1,URS00021F600D_4577,ACTTACCTGGACGGGGTCGACGGGCTATCAAGAAGGCCCGTGGCCT...
13773,1,URS00021F6599_4577,TCTTGCCTTGCGCGATCTAAGTCTAAGGAAATACAAAAATGCGCTT...


## 手动特征

In [47]:
from itertools import product
# compute k-mers features (k=1~4)
def Kmers_funct(seq, x):
    X = [None] * len(seq)
    for i in range(len(seq)):
        a = seq[i]
        t = 0
        l = []
        for index in range(len(a) - x + 1):
            t = a[index:index + x]
            if (len(t)) == x:
                l.append(t)
        X[i] = l
    return X


def nucleotide_type(k):
    z = []
    for i in product('ACGT', repeat=k):
        z.append(''.join(i))
    return z


def Kmers_frequency(seq, x):
    X = []
    char = nucleotide_type(x)
    for i in range(len(seq)):
        s = seq[i]
        frequence = []
        for a in char:
            number = s.count(a)
            char_frequence = number / (len(s) - x + 1)
            frequence.append(char_frequence)
        X.append(frequence)
    return X


feature_1mer = Kmers_funct(data.seq, 1)  # 1-mer
feature_2mer = Kmers_funct(data.seq, 2)  # 2-mer
feature_3mer = Kmers_funct(data.seq, 3)  # 3-mer

feature_1mer_frequency = Kmers_frequency(feature_1mer, 1)  # 1-mer
feature_2mer_frequency = Kmers_frequency(feature_2mer, 2)  # 2-mer
feature_3mer_frequency = Kmers_frequency(feature_3mer, 3)  # 3-mer

feature = pd.concat([pd.DataFrame(feature_1mer_frequency),
                     pd.DataFrame(feature_2mer_frequency),
                     pd.DataFrame(feature_3mer_frequency)], axis=1)
feature.columns = ['A','C','G','T','AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT','AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
feature

,A,C,G,T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.269923,0.229649,0.329906,0.170523,0.081545,0.042060,0.097854,0.048927,0.056652,0.055794,...,0.010318,0.007739,0.018057,0.017197,0.024076,0.006019,0.004299,0.015477,0.006019,0.008598
1,0.135566,0.398724,0.318979,0.146730,0.009600,0.049600,0.048000,0.027200,0.043200,0.158400,...,0.020867,0.011236,0.006421,0.014446,0.019262,0.004815,0.000000,0.014446,0.003210,0.006421
2,0.228774,0.227987,0.272799,0.270440,0.051969,0.048031,0.069291,0.059055,0.069291,0.044882,...,0.013407,0.020505,0.018139,0.026025,0.019716,0.026025,0.011830,0.016562,0.023659,0.021293
3,0.236572,0.266913,0.280033,0.216482,0.044727,0.061551,0.071810,0.058679,0.074682,0.077965,...,0.011910,0.012320,0.021355,0.013142,0.025051,0.015195,0.007803,0.016016,0.015606,0.006571
4,0.312741,0.211068,0.214929,0.261261,0.110968,0.060645,0.073548,0.068387,0.082581,0.051613,...,0.005175,0.023286,0.028461,0.014230,0.020699,0.016818,0.010349,0.025873,0.023286,0.019405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13770,0.271845,0.169903,0.165049,0.393204,0.044118,0.083333,0.044118,0.102941,0.068627,0.014706,...,0.000000,0.014851,0.009901,0.000000,0.034653,0.024752,0.049505,0.009901,0.019802,0.064356
13771,0.346223,0.152878,0.226619,0.274281,0.127928,0.045045,0.079279,0.094595,0.052252,0.027027,...,0.005415,0.011733,0.023466,0.009025,0.015343,0.019856,0.017148,0.012635,0.018953,0.025271
13772,0.177215,0.253165,0.348101,0.221519,0.032051,0.057692,0.051282,0.038462,0.051282,0.083333,...,0.012987,0.006494,0.000000,0.012987,0.058442,0.006494,0.012987,0.006494,0.025974,0.006494
13773,0.314590,0.168693,0.174772,0.341945,0.137195,0.033537,0.044207,0.099085,0.038110,0.044207,...,0.010703,0.027523,0.013761,0.013761,0.015291,0.012232,0.021407,0.024465,0.029052,0.085627


In [48]:
def gc_content(dna_sequence):
    gc_percent = []
    for i in dna_sequence:
        gc_count = i.count('G') + i.count('C')
        total_count = len(i)
        gc_percent_eve = (gc_count / total_count) * 100
        gc_percent.append(gc_percent_eve)
    return gc_percent

In [49]:
# gc_precent = gc_content(data.iloc[:,0])
gc_precent = gc_content(data.seq)
gc_precent_pd = pd.DataFrame(gc_precent)
gc_precent_pd.rename(columns={0: 'GC content'}, inplace=True)
print(gc_precent_pd.shape)
gc_precent_pd

feature.insert(0,'GC content',gc_precent_pd)

feature = pd.DataFrame(feature)
feature.iloc[:,0] = feature.iloc[:,0]/100
feature

(13775, 1)


,GC content,A,C,G,T,AA,AC,AG,AT,CA,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.559554,0.269923,0.229649,0.329906,0.170523,0.081545,0.042060,0.097854,0.048927,0.056652,...,0.010318,0.007739,0.018057,0.017197,0.024076,0.006019,0.004299,0.015477,0.006019,0.008598
1,0.717703,0.135566,0.398724,0.318979,0.146730,0.009600,0.049600,0.048000,0.027200,0.043200,...,0.020867,0.011236,0.006421,0.014446,0.019262,0.004815,0.000000,0.014446,0.003210,0.006421
2,0.500786,0.228774,0.227987,0.272799,0.270440,0.051969,0.048031,0.069291,0.059055,0.069291,...,0.013407,0.020505,0.018139,0.026025,0.019716,0.026025,0.011830,0.016562,0.023659,0.021293
3,0.546945,0.236572,0.266913,0.280033,0.216482,0.044727,0.061551,0.071810,0.058679,0.074682,...,0.011910,0.012320,0.021355,0.013142,0.025051,0.015195,0.007803,0.016016,0.015606,0.006571
4,0.425997,0.312741,0.211068,0.214929,0.261261,0.110968,0.060645,0.073548,0.068387,0.082581,...,0.005175,0.023286,0.028461,0.014230,0.020699,0.016818,0.010349,0.025873,0.023286,0.019405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13770,0.334951,0.271845,0.169903,0.165049,0.393204,0.044118,0.083333,0.044118,0.102941,0.068627,...,0.000000,0.014851,0.009901,0.000000,0.034653,0.024752,0.049505,0.009901,0.019802,0.064356
13771,0.379496,0.346223,0.152878,0.226619,0.274281,0.127928,0.045045,0.079279,0.094595,0.052252,...,0.005415,0.011733,0.023466,0.009025,0.015343,0.019856,0.017148,0.012635,0.018953,0.025271
13772,0.601266,0.177215,0.253165,0.348101,0.221519,0.032051,0.057692,0.051282,0.038462,0.051282,...,0.012987,0.006494,0.000000,0.012987,0.058442,0.006494,0.012987,0.006494,0.025974,0.006494
13773,0.343465,0.314590,0.168693,0.174772,0.341945,0.137195,0.033537,0.044207,0.099085,0.038110,...,0.010703,0.027523,0.013761,0.013761,0.015291,0.012232,0.021407,0.024465,0.029052,0.085627


In [50]:
sequence_len = []
for i in range(data.shape[0]):
    sequence_len.append(len(data.seq[i]))
sequence_len = pd.DataFrame(sequence_len)
sequence_len.rename(columns={0:'seq_len'},inplace=True)
sequence_len

feature.insert(1,'seq_len',sequence_len)
feature.rename(columns={'GC content':'GC_con'},inplace=True)
print(feature.shape)
feature

(13775, 86)


,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.559554,1167,0.269923,0.229649,0.329906,0.170523,0.081545,0.042060,0.097854,0.048927,...,0.010318,0.007739,0.018057,0.017197,0.024076,0.006019,0.004299,0.015477,0.006019,0.008598
1,0.717703,627,0.135566,0.398724,0.318979,0.146730,0.009600,0.049600,0.048000,0.027200,...,0.020867,0.011236,0.006421,0.014446,0.019262,0.004815,0.000000,0.014446,0.003210,0.006421
2,0.500786,1272,0.228774,0.227987,0.272799,0.270440,0.051969,0.048031,0.069291,0.059055,...,0.013407,0.020505,0.018139,0.026025,0.019716,0.026025,0.011830,0.016562,0.023659,0.021293
3,0.546945,2439,0.236572,0.266913,0.280033,0.216482,0.044727,0.061551,0.071810,0.058679,...,0.011910,0.012320,0.021355,0.013142,0.025051,0.015195,0.007803,0.016016,0.015606,0.006571
4,0.425997,777,0.312741,0.211068,0.214929,0.261261,0.110968,0.060645,0.073548,0.068387,...,0.005175,0.023286,0.028461,0.014230,0.020699,0.016818,0.010349,0.025873,0.023286,0.019405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13770,0.334951,206,0.271845,0.169903,0.165049,0.393204,0.044118,0.083333,0.044118,0.102941,...,0.000000,0.014851,0.009901,0.000000,0.034653,0.024752,0.049505,0.009901,0.019802,0.064356
13771,0.379496,1112,0.346223,0.152878,0.226619,0.274281,0.127928,0.045045,0.079279,0.094595,...,0.005415,0.011733,0.023466,0.009025,0.015343,0.019856,0.017148,0.012635,0.018953,0.025271
13772,0.601266,158,0.177215,0.253165,0.348101,0.221519,0.032051,0.057692,0.051282,0.038462,...,0.012987,0.006494,0.000000,0.012987,0.058442,0.006494,0.012987,0.006494,0.025974,0.006494
13773,0.343465,658,0.314590,0.168693,0.174772,0.341945,0.137195,0.033537,0.044207,0.099085,...,0.010703,0.027523,0.013761,0.013761,0.015291,0.012232,0.021407,0.024465,0.029052,0.085627


## cds特征

In [51]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Zea_mays',names=names,sep='\t', index_col=index_col)
rna_cds = rna_cds
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Zea_mays',names=names,sep='\t', index_col=index_col)
rna_cds_nc = rna_cds_nc

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)
rna_cds

data1 = data
data2 = data
for i, l in data2.iterrows():
    data1.iloc[i,2] = len(l['seq'])
    
mylist = []
mylist = pd.DataFrame(mylist,columns=['score','cdsStarts','cdsStop','cdsSizes','cdsPercent'])

for index, row in rna_cds.iterrows():
    tem_pd_score = pd.DataFrame(row)
    mylist.loc[index, 'cdsStarts'] = row["start"]
    mylist.loc[index, 'cdsStop'] = row["end"]
    mylist.loc[index, 'cdsSizes'] = row["end"] - row["start"]
    mylist.loc[index, 'score'] = tem_pd_score.iloc[4, 0]
    mylist.loc[index, 'cdsPercent'] = (row["start"] + row["end"]) / data1.loc[index, 'seq']
    
feature = feature.join(mylist)

# 选取其中10个特征
# feature = feature[['GC_con', 'score', 'cdsStop', 'cdsSizes','T','C','GT','GC','ACG','TAT']]
# 选取其中45个特征
# feature = feature[['GC_con', 'seq_len', 'A', 'C', 'G', 'T', 'AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT', 'AAA', 'AAG', 'AGA', 'ATT', 'CCG', 'CGC', 'CGG', 'CTC', 'GAA', 'GAG', 'GCC', 'GCG', 'GGC', 'GGG', 'TCT', 'TGG', 'TTG', 'TTT', 'score', 'cdsStarts', 'cdsStop', 'cdsSizes', 'cdsPercent']]

feature

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,GGG,TCT,TGG,TTG,TTT,score,cdsStarts,cdsStop,cdsSizes,cdsPercent
0,0.559554,1167,0.269923,0.229649,0.329906,0.170523,0.081545,0.042060,0.097854,0.048927,...,0.021496,0.007739,0.024076,0.006019,0.008598,1367.0,0,1167,1167,1.0
1,0.717703,627,0.135566,0.398724,0.318979,0.146730,0.009600,0.049600,0.048000,0.027200,...,0.019262,0.011236,0.019262,0.003210,0.006421,827.0,0,627,627,1.0
2,0.500786,1272,0.228774,0.227987,0.272799,0.270440,0.051969,0.048031,0.069291,0.059055,...,0.013407,0.020505,0.019716,0.023659,0.021293,1372.0,0,1272,1272,1.0
3,0.546945,2439,0.236572,0.266913,0.280033,0.216482,0.044727,0.061551,0.071810,0.058679,...,0.018070,0.012320,0.025051,0.015606,0.006571,2639.0,0,2439,2439,1.0
4,0.425997,777,0.312741,0.211068,0.214929,0.261261,0.110968,0.060645,0.073548,0.068387,...,0.010349,0.023286,0.020699,0.023286,0.019405,877.0,0,777,777,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13770,0.334951,206,0.271845,0.169903,0.165049,0.393204,0.044118,0.083333,0.044118,0.102941,...,0.004950,0.014851,0.034653,0.019802,0.064356,341.0,60,201,141,1.26699
13771,0.379496,1112,0.346223,0.152878,0.226619,0.274281,0.127928,0.045045,0.079279,0.094595,...,0.017148,0.011733,0.015343,0.018953,0.025271,330.0,238,415,177,0.58723
13772,0.601266,158,0.177215,0.253165,0.348101,0.221519,0.032051,0.057692,0.051282,0.038462,...,0.051948,0.006494,0.058442,0.025974,0.006494,269.0,52,121,69,1.094937
13773,0.343465,658,0.314590,0.168693,0.174772,0.341945,0.137195,0.033537,0.044207,0.099085,...,0.010703,0.027523,0.015291,0.029052,0.085627,257.0,38,95,57,0.202128


## 深度特征

In [52]:
sequences = data3['seq']
labels = data3['label'].values
print(labels)

pat = re.compile('[AGCTagct]')

def pre_process(text):
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(pre_process)

word_set = set()

for lst in x:
    for word in lst:
        word_set.add(word)

word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])

text = x.apply(lambda x: [word_index.get(word, 0) for word in x])
print(text)

text_len = 1000

pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]
pad_text = np.array(pad_text)
pad_text

pad_text,labels=torch.LongTensor(pad_text),torch.LongTensor(labels)
x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)
print(x_train.shape)
print(y_train.shape)

class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list

    def __getitem__(self, index):
        text = torch.LongTensor(self.text_list[index])
        label = self.label_list[index]
        return text, label

    def __len__(self):
        return len(self.text_list)

data_ds = Mydataset(pad_text,labels)
train_ds = Mydataset(x_train, y_train)
test_ds = Mydataset(x_test, y_test)

batch_size = 16

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)
data_dl =torch.utils.data.DataLoader(data_ds,batch_size=batch_size,shuffle=True)

print(len(train_dl))

class PositionalEncoding(nn.Module):
    "Implement the PE function."

#     d_model：表示输入向量的维度（通常也是 Transformer 模型中的隐藏层维度）。
#     dropout：表示应用于位置编码的 dropout 概率。
#     max_len：表示输入文本的最大长度。
    def __init__(self, d_model, dropout=0.1, max_len=text_len) :
        super(PositionalEncoding, self).__init__()
#         实现Dropout正则化减少过拟合
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len,dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# embed_dim = 2 ** (int(np.log2(len(word_list) ** 0.25)) + 2)  # 经验值

embedding_dim = 50
print('embedding_dim:', embedding_dim)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.em = nn.Embedding(len(word_list) + 1, embedding_dim=50)  # 对0也需要编码
        self.pos = PositionalEncoding(embedding_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(embedding_dim, nhead=5)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)
        self.pool1 = nn.MaxPool1d(5)
        self.pool2 = nn.MaxPool1d(3)
        self.fc1 = nn.Linear(embedding_dim, 128)
        self.fc2 = nn.Linear(128, 2)


    def forward(self, inputs):
        x = self.em(inputs)                                  #[16,1000,50]
        x = self.pos(x)                                     #[16,1000,50]
        x = x.float()                                       #[16,1000,50]
        x = self.transformer_encoder(x)                     #[16,1000,50]
        x = self.pool1(x)
        x = self.pool2(x)
        x = x.view(-1, 3000)  
        return x

model = Net()
model = model.to(device)

loss = nn.CrossEntropyLoss()
loss = loss.to(device)
criterion=torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#提取深度模型的特征
model=model

with torch.no_grad():
    outputs=[]
    for x,y in data_dl:
        x.to('cuda:0')
        out = model(x)
        outputs.append(out)

outputs1 = torch.concat(outputs,axis =0)

deep_fea =pd.DataFrame(outputs1)
deep_fea.to_csv(r"C:\Users\26970\Desktop\特征融合\45特征\Zea_mays-transformer.csv")

[0 0 0 ... 1 1 1]
0        [3, 4, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 3, 1, 3, ...
1        [3, 4, 2, 2, 1, 1, 4, 1, 4, 1, 1, 4, 4, 1, 1, ...
2        [3, 4, 2, 4, 4, 4, 1, 4, 4, 3, 3, 2, 2, 3, 3, ...
3        [3, 4, 2, 2, 1, 4, 1, 3, 2, 3, 4, 1, 4, 4, 2, ...
4        [3, 4, 2, 1, 3, 1, 1, 3, 2, 2, 3, 4, 2, 1, 3, ...
                               ...                        
13770    [3, 4, 3, 1, 3, 4, 4, 4, 1, 4, 4, 2, 2, 1, 1, ...
13771    [2, 2, 2, 2, 3, 3, 3, 2, 4, 1, 4, 4, 4, 2, 3, ...
13772    [3, 1, 4, 4, 3, 1, 1, 4, 2, 2, 3, 1, 2, 2, 2, ...
13773    [4, 1, 4, 4, 2, 1, 1, 4, 4, 2, 1, 2, 1, 2, 3, ...
13774    [3, 1, 1, 4, 2, 2, 3, 1, 2, 4, 3, 2, 1, 2, 3, ...
Name: seq, Length: 13775, dtype: object
torch.Size([9642, 1000])
torch.Size([9642])
603
embedding_dim: 50


In [53]:
fea_all = feature.join(deep_fea)
fea_all

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.559554,1167,0.269923,0.229649,0.329906,0.170523,0.081545,0.042060,0.097854,0.048927,...,1.837314,1.450798,1.534154,1.559816,1.232797,1.673976,1.178450,1.702446,1.680991,1.702748
1,0.717703,627,0.135566,0.398724,0.318979,0.146730,0.009600,0.049600,0.048000,0.027200,...,0.913202,2.192281,1.336017,1.105397,2.434132,1.437545,1.522616,2.046743,0.671827,1.403665
2,0.500786,1272,0.228774,0.227987,0.272799,0.270440,0.051969,0.048031,0.069291,0.059055,...,1.389281,2.101300,0.863084,1.042309,1.880923,0.995786,1.221520,2.642556,1.750257,1.646719
3,0.546945,2439,0.236572,0.266913,0.280033,0.216482,0.044727,0.061551,0.071810,0.058679,...,1.423985,1.987946,1.187338,1.188717,2.066113,1.595048,1.196837,2.483173,1.192928,1.854896
4,0.425997,777,0.312741,0.211068,0.214929,0.261261,0.110968,0.060645,0.073548,0.068387,...,1.352524,1.954160,1.740655,1.282911,1.989328,0.874023,0.952622,2.276921,1.454377,1.515607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13770,0.334951,206,0.271845,0.169903,0.165049,0.393204,0.044118,0.083333,0.044118,0.102941,...,1.137824,2.595189,1.763927,1.047071,2.662140,1.475130,1.245429,1.539945,1.628802,1.695455
13771,0.379496,1112,0.346223,0.152878,0.226619,0.274281,0.127928,0.045045,0.079279,0.094595,...,1.419003,1.576783,1.197084,2.299988,1.582089,1.610765,1.789119,2.447420,2.104601,0.847916
13772,0.601266,158,0.177215,0.253165,0.348101,0.221519,0.032051,0.057692,0.051282,0.038462,...,1.778244,2.003427,2.169198,2.342249,1.446033,1.644490,1.312985,2.203384,1.743354,1.714045
13773,0.343465,658,0.314590,0.168693,0.174772,0.341945,0.137195,0.033537,0.044207,0.099085,...,1.416774,1.710226,1.873754,1.573585,2.038150,1.947701,1.818329,1.775032,1.959071,1.296122


In [54]:
y = pd.DataFrame(label_data, columns=['label'])

fea_all.insert(0,'label',y)
fea_all

,label,GC_con,seq_len,A,C,G,T,AA,AC,AG,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,0.559554,1167,0.269923,0.229649,0.329906,0.170523,0.081545,0.042060,0.097854,...,1.837314,1.450798,1.534154,1.559816,1.232797,1.673976,1.178450,1.702446,1.680991,1.702748
1,0,0.717703,627,0.135566,0.398724,0.318979,0.146730,0.009600,0.049600,0.048000,...,0.913202,2.192281,1.336017,1.105397,2.434132,1.437545,1.522616,2.046743,0.671827,1.403665
2,0,0.500786,1272,0.228774,0.227987,0.272799,0.270440,0.051969,0.048031,0.069291,...,1.389281,2.101300,0.863084,1.042309,1.880923,0.995786,1.221520,2.642556,1.750257,1.646719
3,0,0.546945,2439,0.236572,0.266913,0.280033,0.216482,0.044727,0.061551,0.071810,...,1.423985,1.987946,1.187338,1.188717,2.066113,1.595048,1.196837,2.483173,1.192928,1.854896
4,0,0.425997,777,0.312741,0.211068,0.214929,0.261261,0.110968,0.060645,0.073548,...,1.352524,1.954160,1.740655,1.282911,1.989328,0.874023,0.952622,2.276921,1.454377,1.515607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13770,1,0.334951,206,0.271845,0.169903,0.165049,0.393204,0.044118,0.083333,0.044118,...,1.137824,2.595189,1.763927,1.047071,2.662140,1.475130,1.245429,1.539945,1.628802,1.695455
13771,1,0.379496,1112,0.346223,0.152878,0.226619,0.274281,0.127928,0.045045,0.079279,...,1.419003,1.576783,1.197084,2.299988,1.582089,1.610765,1.789119,2.447420,2.104601,0.847916
13772,1,0.601266,158,0.177215,0.253165,0.348101,0.221519,0.032051,0.057692,0.051282,...,1.778244,2.003427,2.169198,2.342249,1.446033,1.644490,1.312985,2.203384,1.743354,1.714045
13773,1,0.343465,658,0.314590,0.168693,0.174772,0.341945,0.137195,0.033537,0.044207,...,1.416774,1.710226,1.873754,1.573585,2.038150,1.947701,1.818329,1.775032,1.959071,1.296122


In [112]:
# 定义朴素贝叶斯模型
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict

def model_gnb(X, y, X_test, y_test):
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    gnb = GaussianNB()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行五倍交叉验证
    cv_results = cross_validate(gnb, X, y, scoring=scoring, cv=5, return_train_score=False)

    # 计算并输出评估指标
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(gnb, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")
    
    # 在独立测试集上评估模型
    gnb.fit(X, y)
    y_pred_test = gnb.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")

model_gnb(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.784188
F1 score: 0.771896
ROC AUC score: 0.880872
MCC: 0.573147


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 0.763839
PPV: 0.807937
Sensitivity (Recall): 0.745625
Specificity: 0.822750


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.859375
Test Set - F1 score: 0.875229
Test Set - ROC AUC score: 0.860317
Test Set - MCC: 0.746533
Test Set - NPV: 0.991221
Test Set - PPV: 0.782141
Test Set - Sensitivity (Recall): 0.993468
Test Set - Specificity: 0.727166


In [55]:
# 定义SVM模型

def model_SVM(X, y, X_test, y_test):
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建SVM分类器
    svm_clf = SVC()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(svm_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(svm_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    svm_clf.fit(X, y)
    y_pred_test = svm_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_SVM(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.928125
F1 score: 0.933050
ROC AUC score: 0.967438
MCC: 0.865402


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 1.000000
PPV: 0.874317
Sensitivity (Recall): 1.000000
Specificity: 0.856250


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.920290
Test Set - F1 score: 0.919819
Test Set - ROC AUC score: 0.924936
Test Set - MCC: 0.849187
Test Set - NPV: 0.987767
Test Set - PPV: 0.860735
Test Set - Sensitivity (Recall): 0.987612
Test Set - Specificity: 0.862260


In [56]:
### RFC训练模型

def model_RFC(X, y, X_test, y_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict

    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建随机森林分类器
    rf_clf = RandomForestClassifier()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(rf_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")
    
    # 在独立测试集上评估模型
    rf_clf.fit(X, y)
    y_pred_test = rf_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")

model_RFC(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarni

Accuracy: 0.939688
F1 score: 0.943134
ROC AUC score: 0.964381
MCC: 0.885827


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionW

NPV: 0.999006
PPV: 0.892076
Sensitivity (Recall): 0.999125
Specificity: 0.879125


C:\Users\26970\AppData\Local\Temp\ipykernel_29536\1020548416.py:46: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X, y)


Test Set - Accuracy: 0.926243
Test Set - F1 score: 0.921314
Test Set - ROC AUC score: 0.926691
Test Set - MCC: 0.852151
Test Set - NPV: 0.940746
Test Set - PPV: 0.910176
Test Set - Sensitivity (Recall): 0.932727
Test Set - Specificity: 0.920654
